In [53]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

In [54]:
demand = {
    "Kunde 1": 62,
    "Kunde 2": 83,
    "Kunde 3": 39,
    "Kunde 4": 91,
}
customers = demand.keys()

supply = {
    "Fabrik 1": 135,
    "Fabrik 2": 56,
    "Fabrik 3": 93
}
factories = supply.keys()

costs = {
    ("Fabrik 1", "Kunde 1"): 132,
    ("Fabrik 1", "Kunde 3"): 97,
    ("Fabrik 1", "Kunde 4"): 103,
    ("Fabrik 2", "Kunde 1"): 85,
    ("Fabrik 2", "Kunde 2"): 91,
    ("Fabrik 3", "Kunde 1"): 106,
    ("Fabrik 3", "Kunde 2"): 89,
    ("Fabrik 3", "Kunde 3"): 100,
    ("Fabrik 3", "Kunde 4"): 89,
}

In [55]:
# Create a new model
m = gp.Model("transportation")

# Create variables
flow = m.addVars(costs.keys(), name="flow")
flow

{('Fabrik 1', 'Kunde 1'): <gurobi.Var *Awaiting Model Update*>,
 ('Fabrik 1', 'Kunde 3'): <gurobi.Var *Awaiting Model Update*>,
 ('Fabrik 1', 'Kunde 4'): <gurobi.Var *Awaiting Model Update*>,
 ('Fabrik 2', 'Kunde 1'): <gurobi.Var *Awaiting Model Update*>,
 ('Fabrik 2', 'Kunde 2'): <gurobi.Var *Awaiting Model Update*>,
 ('Fabrik 3', 'Kunde 1'): <gurobi.Var *Awaiting Model Update*>,
 ('Fabrik 3', 'Kunde 2'): <gurobi.Var *Awaiting Model Update*>,
 ('Fabrik 3', 'Kunde 3'): <gurobi.Var *Awaiting Model Update*>,
 ('Fabrik 3', 'Kunde 4'): <gurobi.Var *Awaiting Model Update*>}

In [56]:
# Set objective
m.setObjective(flow.prod(costs), GRB.MINIMIZE)

In [57]:
# Constraints
# Can't exceed supply
m.addConstrs((gp.quicksum(flow.select(f, "*")) <= supply[f] for f in factories), name="supply")

# Must meet demand
m.addConstrs((gp.quicksum(flow.select("*", c)) == demand[c] for c in customers), name="demand")

{'Kunde 1': <gurobi.Constr *Awaiting Model Update*>,
 'Kunde 2': <gurobi.Constr *Awaiting Model Update*>,
 'Kunde 3': <gurobi.Constr *Awaiting Model Update*>,
 'Kunde 4': <gurobi.Constr *Awaiting Model Update*>}

In [58]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 5900X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Academic license 2495314 - for non-commercial use only - registered to re___@h-ka.de
Optimize a model with 7 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xe00b8a58
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolved: 4 rows, 5 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4888062e+04   8.501250e+00   0.000000e+00      0s
       3    2.5883000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.588300000e+04


In [59]:
# Print solution
if m.status == GRB.OPTIMAL:
    solution = pd.DataFrame(columns=customers, index=factories, data=0.0)
    for f, c in flow.keys():
        solution.loc[f, c] = flow[f, c].x
    print(solution)
# Total transportation cost
print("\nTotal cost: ", m.objVal)

          Kunde 1  Kunde 2  Kunde 3  Kunde 4
Fabrik 1      0.0      0.0     39.0     87.0
Fabrik 2     56.0      0.0      0.0      0.0
Fabrik 3      6.0     83.0      0.0      4.0

Total cost:  25883.0
